## Import

In [163]:
from tqdm import tqdm

import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import re


## Load data

In [164]:
data_path = "C:\\Users\\PC0\\Documents\\GitHub\\Korea_Investment_and_Securities\\datasets\\"

In [117]:
am_stocks = pd.read_csv(data_path + "amrican_ticker_stockname.csv")
info_df = pd.read_csv(data_path + "info_df.csv")
tot_df = pd.read_csv(data_path + "tot_df.csv")
final_df = pd.read_csv(data_path + "all_joined_stocks.csv")
etf_niche = pd.read_csv(data_path + "etf_niche_grouped_df.csv")

In [21]:
info_df.rename(columns = {"Unnamed: 0":"Symbol"}, inplace = True)

In [26]:
cap_df = info_df.loc[:, ['Symbol', 'marketCap']]

In [27]:
joined_df = pd.merge(am_stocks, cap_df, how = "inner")

In [29]:
joined_df.head()

,Symbol,Name,marketCap
0,NVDA,NVIDIA Corp,6.047336e+11
1,QCOM,Qualcomm Inc,1.849456e+11
2,NXPI,NXP Semiconductors NV,4.943163e+10
3,AMD,Advanced Micro Devices Inc,1.379936e+11
4,MRVL,Marvell Technology Inc,5.716745e+10


In [30]:
# joined_df.to_csv(data_path + "marketCap_joined_df.csv", index = False)

In [43]:
final_df.drop_duplicates(inplace = True)

In [77]:
# ticker별 최소 길이 stock names 탐색
for idx, ticker in enumerate(final_df.Ticker.unique()):
    temp_df = final_df.loc[final_df.Ticker == ticker]
    len_list = np.array([len(x) for x in temp_df.stock_names])
    min_idx = np.argmin(len_list)
    
    if idx == 0:
        output_df = pd.DataFrame(temp_df.iloc[min_idx,:]).T
    else:
        output_df = output_df.append(pd.DataFrame(temp_df.iloc[min_idx,:]).T)

In [83]:
# output_df.to_csv(data_path + "ticker_stocks_df.csv", index = False)

In [101]:
temp_df = pd.merge(cap_df, output_df.loc[:, ['Ticker', "stock_names"]], left_on = 'Symbol', right_on = 'Ticker', how = "inner")

del temp_df['Ticker']

temp_df = temp_df.loc[:, ['Symbol', 'stock_names', 'marketCap']]
theme_df = pd.merge(etf_niche, temp_df, on = "stock_names", how = "outer")
theme_df = theme_df.loc[:, ['Theme', 'Symbol', 'stock_names', 'etf_names', 'marketCap']]

In [102]:
theme_df.head()

,Theme,Symbol,stock_names,etf_names,marketCap
0,5G,NVDA,NVIDIA Corporation,4,6.047336e+11
1,Agriculture,NVDA,NVIDIA Corporation,1,6.047336e+11
2,Blockchain,NVDA,NVIDIA Corporation,5,6.047336e+11
3,Broad Technology,NVDA,NVIDIA Corporation,3,6.047336e+11
4,Broad Thematic,NVDA,NVIDIA Corporation,11,6.047336e+11


In [105]:
# 최종 데이터 합본
for idx, theme in enumerate(theme_df.Theme.unique()):
    temp_df2 = theme_df.loc[theme_df.Theme == theme, :]
    
    if idx == 0:
        final_df2 = temp_df2.sort_values("etf_names", ascending = False)
    else:
        final_df2 = final_df2.append(temp_df2.sort_values("etf_names", ascending = False))

In [114]:
final_df2.rename(columns = {"etf_names":"cnts"}, inplace = True)
# final_df2.to_csv(data_path + "etf_niche_grouped_df2.csv", index = False)

In [165]:
final_df2 = pd.read_csv(data_path + "etf_niche_grouped_df2.csv")

In [166]:
final_df2.head(20)

,Theme,Symbol,stock_names,cnts,marketCap
0,5G,NVDA,NVIDIA Corporation,4,6.047336e+11
1,5G,NXPI,NXP Semiconductors NV,3,4.943163e+10
2,5G,AMD,"Advanced Micro Devices, Inc.",3,1.379936e+11
3,5G,MRVL,"Marvell Technology, Inc.",3,5.716745e+10
4,5G,NaN,"Amazon.com, Inc.",3,NaN
5,5G,AAPL,Apple Inc.,3,2.770713e+12
6,5G,QCOM,Qualcomm Inc,3,1.849456e+11
7,5G,NaN,"Analog Devices, Inc.",2,NaN
8,5G,NaN,MediaTek Inc,2,NaN
9,5G,FB,Meta Platforms Inc. Class A,2,6.055891e+11


In [173]:
import re

In [233]:
unique_list = final_df2.stock_names.unique()

In [234]:
after_list = [x for x in unique_list if ('-' not in x) and ('%' not in x)]

In [229]:
after_list = [re.sub(" Class A", "", x) for x in np.unique(after_list)]

In [230]:
after_list = [re.sub(" Class B", "", x) for x in np.unique(after_list)]

In [236]:
np.save(data_path + "stock_names.npy",np.unique(after_list))

In [232]:
np.unique(after_list).shape

(10055,)

In [157]:
change_dict = {}

In [159]:
for fdr_name in tot_df.Name:
    empty_list = []
    for etf_name in final_df2.stock_names.unique():
        if fdr_name in etf_name:
            empty_list.append(etf_name)
    change_dict[fdr_name] = empty_list